Transfer_Xception.ipynb

Julia Goyco, Zoe Lambert, Jennifer Mince, Stephanie Schoch

COSC480 - Major Project: Image Classification of Mixed Dog Breeds

This project aims to create a deep neural network with several goals.
First, we want to accurately predict dog breeds. Second, we want to extend
this to the application of mixed dog breeds. Specifically,
we want to identify the parent breeds in known mixed dog breeds. This involves
the creation of a mixed-breed dog dataset and the application of transfer learning
techniques to the mixed-breed dataset.

Need to install: tqdm (Anaconda library) - progress meter
Also need: Mixed Breed dataset from Google Project Folder. This file is too large to upload to github.
Resources for running are in Google folders Mixed-Breed Dog Images; resources for running code

Project due date: 12/10/18

# Transfer Learning with keras pretrained model- Xception 
This model has weights that have been pretrained on imagenet
More layers are added on top of the model.
Could use data augmentation for better results- I did not try that yet
This still need to be fine-tuned for better performance

There are other pretrained models we can try if this doesnt seem like it is working

Another thing that could be done is geting all the mixed dog breed images to arrays

Here are some links 
https://www.depends-on-the-definition.com/transfer-learning-for-dog-breed-identification/

https://github.com/booleanhunter/Deep-Learning-Projects/blob/master/Predict%20Dog%20Breeds/dog_app.ipynb


In [ ]:
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras.models import Model, Input
from keras.preprocessing import image
from keras.applications import xception
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint

# For Dog Breed Predictor
from scipy import ndimage, io, misc
from xml.dom import minidom
from matplotlib.pyplot import imshow        
import matplotlib.pyplot as plt  
import imageio
%matplotlib inline                          
import pandas as pd
from keras.applications.xception import Xception, preprocess_input, decode_predictions

Download data from google drive folder and save in the same directory as this notebook

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# Load image and label data.
X = np.load(open('Dog_images.npy','rb'))
Y = np.load(open('Dog_labels.npy','rb'))

pre_y = preprocessing.minmax_scale(Y, feature_range=(0,119))

y_cat = to_categorical(pre_y, num_classes=None)

#Split between the train and test data.
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.20, random_state=seed)

Download DogXceptionData.npz from link google drive folder

In [ ]:
#Load bottleneck features from DogXceptionData.npz. 
#These will serve as the last activation maps prior to the fully connected layers.
bottleneck_features = np.load('DogXceptionData.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']

In [ ]:
# Using the Xception model, pretrained on ImageNet database.
# Xception: better performance over Inception by using depthwise separable convolutions.
def extract_Xception(tensor):
    return Xception(weights='imagenet', include_top=False).predict(preprocess_input(tensor))

On top of the pretrained model a fully connected layer with 1024 neurons and some Dropout is added.

In [ ]:
NUM_CLASSES = 120
# create the base pre-trained model
base_model = xception.Xception(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output

#Can try adding more layers
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

# Add logistic layer with NUM_CLASSES.
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# checkpoints: to save the best weights (only if accuracy improves)
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

Can try fine tuning more layers
Can try using different optimizers

In [ ]:
# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers[:116]:
     layer.trainable = False
for layer in base_model.layers[116:]:
     layer.trainable = True



# compile the model (should be done *after* setting layers to non-trainable)
#Can try different optimizers like adam
optimizer = RMSprop(lr=0.001, rho=0.9)

# Load weights from file in local directory (saved at previous checkpoint)
# in example at https://machinelearningmastery.com/check-point-deep-learning-models-keras/
# they used binary_crossentropy, but this shouldn't affect results.
#model.load_weights("weights.best.hdf5")

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])

In [ ]:
# train the model on the new data for a few epochs
BATCH_SIZE = 32
EPOCHS = 4

#Checkpoints still need to be added to model.fit call
hist = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=EPOCHS, 
          batch_size=BATCH_SIZE, callbacks=callbacks_list,verbose = 1)

In [ ]:
bst_val_acc = max(hist.history['val_acc'])
print("Best val acc: {:.1%}".format(bst_val_acc))

In [ ]:
#checkpoint, load in best weights
filename = "Dev cloud\weights.best.hdf5"
model.load_weights(filename)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])

scores = model.evaluate(X_test, y_test, verbose=2)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Mixed Dog Breed Transfer Learning

In [ ]:
#Mixed dog breed data

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load data
X = np.load(open('mixed breed\Mixed_train_images.npy','rb'))
Y = np.load(open('mixed breed\Mixed_train_labels.npy','rb'))

pre_y = preprocessing.minmax_scale(Y, feature_range=(0,119))

y_cat = to_categorical(pre_y, num_classes=None)

X_mixed_train, X_mixed_test, y_mixed_train, y_mixed_test = train_test_split(X, y_cat, test_size=0.20, random_state=seed)

In [ ]:
# checkpoints: to save the best weights (only if accuracy improves)
filepath="mixed_weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]



In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
for layer in base_model.layers:
    layer.trainable = False


# compile the model (should be done *after* setting layers to non-trainable)
#Can try different optimizers like adam
optimizer = RMSprop(lr=0.001, rho=0.9)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])



In [ ]:
# train the model on the new data for a few epochs
BATCH_SIZE = 32
EPOCHS = 1

#Checkpoints still need to be added to model.fit call
hist = model.fit(X_mixed_train, y_mixed_train, validation_data=(X_mixed_test, y_mixed_test),epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks_list)


In [ ]:
bst_val_acc = max(hist.history['val_acc'])
print("Best val acc: {:.1%}".format(bst_val_acc))

In [ ]:
#Load in best weights from checkpoint
filename = "Dev cloud\mixed_weights.best.hdf5"
model.load_weights(filename)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])




In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

# Mixed Dog Breed Predictor

In [ ]:
#read labels.csv 
data = pd.read_csv("labels.csv") 

pic_breed = data["breed"]

#Get dog names
#creates a list of all mixed dog breeds
unique_Breed = pic_breed.unique()

In [ ]:
def path_to_tensor(img_path):
    # Loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224))
    
    # Convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    
    # Convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [ ]:
def display_image(img_path):
    a = imageio.imread(img_path)[:,:,:3]
    imshow(a)

In [ ]:
#predicts a vector based on an image
def Mixed_detect_dog(model, image_path):
    tensor = path_to_tensor(image_path)
    bottleneck_features = extract_Xception(tensor)
    predicted_vector = model.predict(tensor)
    predicted_indices = np.argsort(predicted_vector)
    flat = predicted_indices.flatten()
    first_predict = unique_Breed[flat[-1]]
    second_predict = unique_Breed[flat[-2]]
    return first_predict,second_predict

In [ ]:
def Mixed_dog_prediction(img_path):   
    display_image(img_path)
    prediction1,prediction2  = Mixed_detect_dog(model, img_path)
    print('This dog is a mix of {} and {} ! '.format(prediction1,prediction2))


In [ ]:
Mixed_dog_prediction("n02118010_8.jpg")

In [ ]:
Mixed_dog_prediction("n02110000_3.jpg")